In [ ]:
# Copyright 2021 NVIDIA CORPORATION
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import glob
import pandas as pd
import numpy as np
import cudf
import cupy
import gc

from datetime import datetime

In [2]:
files = glob.glob('/raid/recsys2021_pre/*')

In [3]:
files

['/raid/recsys2021_pre/part-00224.parquet',
 '/raid/recsys2021_pre/part-00064.parquet',
 '/raid/recsys2021_pre/part-00061.parquet',
 '/raid/recsys2021_pre/part-00057.parquet',
 '/raid/recsys2021_pre/part-00039.parquet',
 '/raid/recsys2021_pre/part-00164.parquet',
 '/raid/recsys2021_pre/part-00101.parquet',
 '/raid/recsys2021_pre/part-00141.parquet',
 '/raid/recsys2021_pre/part-00040.parquet',
 '/raid/recsys2021_pre/part-00119.parquet',
 '/raid/recsys2021_pre/part-00154.parquet',
 '/raid/recsys2021_pre/part-00163.parquet',
 '/raid/recsys2021_pre/part-00060.parquet',
 '/raid/recsys2021_pre/part-00118.parquet',
 '/raid/recsys2021_pre/part-00136.parquet',
 '/raid/recsys2021_pre/part-00144.parquet',
 '/raid/recsys2021_pre/part-00062.parquet',
 '/raid/recsys2021_pre/part-00210.parquet',
 '/raid/recsys2021_pre/part-00029.parquet',
 '/raid/recsys2021_pre/part-00059.parquet',
 '/raid/recsys2021_pre/part-00198.parquet',
 '/raid/recsys2021_pre/part-00183.parquet',
 '/raid/recsys2021_pre/part-0022

In [4]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"

In [5]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [6]:
cluster = LocalCUDACluster(local_directory='/raid/dask2/', device_memory_limit=0.5)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:33143 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 429.50 GB


In [7]:
ddf = dask_cudf.read_parquet(files)

In [8]:
meta = ('time', 'datetime64[s]')

def convert_timestamp(x):
    x = cudf.to_datetime(x, unit='s')
    return(x)

ddf['date'] = ddf['timestamp'].map_partitions(convert_timestamp, meta=meta)

In [9]:
VALID_DOW = '2021-02-18'
if VALID_DOW!='':
    print('True')
    train = ddf[ddf['date']<cudf.to_datetime(VALID_DOW)].reset_index(drop=True)
else:
    print('False')
    train = ddf

True


In [10]:
train.head()

,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,tw_word4,tw_tweet,group,dt_day,dt_dow,dt_minute,len_hashtags,len_links,len_domains,date
0,606344963,-7651340863584439249,1,0,0,1,1,1612712747,-1617640681192103974,526918,...,0,1716595265,4,7,6,945,1,0,0,2021-02-07 15:45:47
1,0,-3385391863863081084,0,0,0,2,0,1612452111,174132517510076348,2298,...,1808297570,2059407237,3,4,3,921,0,0,0,2021-02-04 15:21:51
2,0,4878369566409755704,1,0,0,0,0,1612980390,160295098824211933,4813450,...,53050233,332220573,4,10,2,1086,0,0,0,2021-02-10 18:06:30
3,-1439756462,6645932351682048504,0,0,0,1,0,1612834174,7488217763927528828,2740151,...,-108567334,-1865257506,4,9,1,89,1,0,0,2021-02-09 01:29:34
4,1461451939,6659868939833331569,1,0,0,0,4,1613574823,-5127534676538097260,115,...,0,-773512424,0,17,2,913,2,0,0,2021-02-17 15:13:43


In [11]:
train.columns

Index(['hashtags', 'tweet_id', 'media', 'links', 'domains', 'tweet_type',
       'language', 'timestamp', 'a_user_id', 'a_follower_count',
       'a_following_count', 'a_is_verified', 'a_account_creation', 'b_user_id',
       'b_follower_count', 'b_following_count', 'b_is_verified',
       'b_account_creation', 'b_follows_a', 'reply', 'retweet',
       'retweet_comment', 'like', 'tw_len_media', 'tw_len_photo',
       'tw_len_video', 'tw_len_gif', 'tw_len_quest', 'tw_len_token',
       'tw_count_capital_words', 'tw_count_excl_quest_marks',
       'tw_count_special1', 'tw_count_hash', 'tw_last_quest', 'tw_len_retweet',
       'tw_len_rt', 'tw_count_at', 'tw_count_words', 'tw_count_char',
       'tw_rt_count_words', 'tw_rt_count_char', 'tw_original_user0',
       'tw_original_user1', 'tw_original_user2', 'tw_rt_user0',
       'tw_original_http0', 'tw_word0', 'tw_word1', 'tw_word2', 'tw_word3',
       'tw_word4', 'tw_tweet', 'group', 'dt_day', 'dt_dow', 'dt_minute',
       'len_hashtags', 

In [12]:
#!rm -r /raid/TE/
#!mkdir /raid/TE/

In [13]:
means = {}
means['reply'] = 0.02846728456689906
means['like'] = 0.3968895210408169
means['retweet'] = 0.08769760903336701
means['retweet_comment'] = 0.006918407917391091
psmooth = 20

In [15]:
TE_cols = [
    ['a_user_id'],
    ['domains', 'language', 'b_follows_a', 'tweet_type', 'media', 'a_is_verified'],
    ['media', 'tweet_type', 'language'],
    ['media', 'tweet_type', 'language', 'a_is_verified', 'b_is_verified', 'b_follows_a'],
    ['b_user_id'],
    ['b_is_verified', 'tweet_type'],
    ['tweet_type'],
    ['tw_original_user0', 'tweet_type', 'language'],
    ['tw_original_user1', 'tweet_type', 'language']
]

In [16]:
%%time

for col in TE_cols:
    print(col)
    df_tmp = train[
        col + ['reply', 'retweet', 'retweet_comment', 'like']
         ].groupby(col).agg({
        'reply': ['sum', 'count'],
        'retweet': ['sum'], 
        'retweet_comment': ['sum'], 
        'like': ['sum']}
    )
    df_tmp = df_tmp.reset_index()
    df_tmp.columns = col + [
        'reply_sum',
        'reply_count',
        'retweet_sum',
        'retweet_comment_sum',
        'like_sum']
    if False:
        print('Filter>4')
        df_tmp = df_tmp[df_tmp['reply_count']>4]
        df_tmp, = dask.persist(df_tmp)
        n_len = df_tmp.shape[0].compute()
        if n_len>3000000:
            print('Filter std')
            for key in means.keys():
                df_tmp['TE_' + key] = (df_tmp[key + '_sum']+means[key]*psmooth)/(df_tmp['reply_count']+psmooth)
                df_tmp['diff_mean' + key] = (df_tmp['TE_' + key]-means[key]).abs()
            df_tmp = df_tmp[((df_tmp['diff_meanreply']>0.03)|(df_tmp['diff_meanlike']>0.075)|
                             (df_tmp['diff_meanretweet']>0.055)|(df_tmp['diff_meanretweet_comment']>0.011))]
    df_tmp = df_tmp[col + [
        'reply_sum',
        'reply_count',
        'retweet_sum',
        'retweet_comment_sum',
        'like_sum'
    ]].to_parquet('/raid/TE/' + '_'.join(col) + '.parquet')

['media', 'tweet_type', 'language', 'a_is_verified', 'b_is_verified', 'b_follows_a', 'tw_last_quest']
['media', 'tweet_type', 'language', 'a_is_verified', 'b_is_verified', 'b_follows_a', 'tw_len_rt']
['tw_len_rt', 'tweet_type', 'language']
['tw_len_rt', 'tweet_type', 'language', 'tw_last_quest']
['tw_len_rt', 'tweet_type', 'language', 'tw_len_quest']
['tw_last_quest', 'tweet_type', 'language']
['tw_len_quest', 'tweet_type', 'language']
['tw_original_user0', 'tw_len_quest', 'tweet_type', 'language']
['tw_original_user0', 'tw_last_quest', 'tweet_type', 'language']
['tw_len_retweet', 'tweet_type', 'language']
['tw_len_retweet', 'tw_len_rt', 'tweet_type', 'language']
['tw_rt_user0', 'b_user_id']


MemoryError: std::bad_alloc: CUDA error at: /opt/conda/envs/rapids/include/rmm/mr/device/cuda_memory_resource.hpp:69: cudaErrorMemoryAllocation out of memory

In [17]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import glob
import pandas as pd
import numpy as np
import gc

from datetime import datetime

files = glob.glob('/raid/recsys2021_pre/*')

import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client, wait

client = Client(local_directory='/raid/dask342/',
                n_workers=8, 
                threads_per_worker=1,
                memory_limit='300GB')

ddf = dd.read_parquet(files, columns=['b_user_id', 'a_user_id', 'tweet_type',  
                                      'language', 'timestamp'] + ['reply', 'retweet', 'retweet_comment', 'like'])

meta = ('time', 'datetime64[s]')

def convert_timestamp(x):
    x = pd.to_datetime(x, unit='s')
    return(x)

ddf['date'] = ddf['timestamp'].map_partitions(convert_timestamp)

VALID_DOW = '2021-02-18'
if VALID_DOW!='':
    print('True')
    train = ddf[ddf['date']<pd.to_datetime(VALID_DOW)].reset_index(drop=True)
else:
    print('False')
    train = ddf
    
means = {}
means['reply'] = 0.02846728456689906
means['like'] = 0.3968895210408169
means['retweet'] = 0.08769760903336701
means['retweet_comment'] = 0.006918407917391091
psmooth = 20

TE_cols = [
    #['b_user_id', 'tweet_type', 'language'],
    ['b_user_id', 'a_user_id']
]

for col in TE_cols:
    print(col)
    df_tmp = train[
        col + ['reply', 'retweet', 'retweet_comment', 'like']
         ].groupby(col).agg({
        'reply': ['sum', 'count'],
        'retweet': ['sum'], 
        'retweet_comment': ['sum'], 
        'like': ['sum']}
    )
    df_tmp = df_tmp.reset_index()
    df_tmp.columns = col + [
        'reply_sum',
        'reply_count',
        'retweet_sum',
        'retweet_comment_sum',
        'like_sum']
    if False:
        print('Filter>4')
        df_tmp = df_tmp[df_tmp['reply_count']>4]
        df_tmp, = dask.persist(df_tmp)
        n_len = df_tmp.shape[0].compute()
        if n_len>3000000:
            print('Filter std')
            for key in means.keys():
                df_tmp['TE_' + key] = (df_tmp[key + '_sum']+means[key]*psmooth)/(df_tmp['reply_count']+psmooth)
                df_tmp['diff_mean' + key] = (df_tmp['TE_' + key]-means[key]).abs()
            df_tmp = df_tmp[((df_tmp['diff_meanreply']>0.03)|(df_tmp['diff_meanlike']>0.075)|
                             (df_tmp['diff_meanretweet']>0.055)|(df_tmp['diff_meanretweet_comment']>0.011))]
    df_tmp = df_tmp[col + [
        'reply_sum',
        'reply_count',
        'retweet_sum',
        'retweet_comment_sum',
        'like_sum'
    ]].to_parquet('/raid/TE/' + '_'.join(col) + '.parquet')

True
['b_user_id', 'a_user_id']
